# Loggi Deliveries Project
## Data

This project aims to analyse the data from the repository [Loggi Benchmark for Urban Deliveries (BUD)](https://github.com/loggi/loggibud). In it, there is data in the JSON format describing the names and location of transportation centers of the company, including the geografical data (latitude and longitude), and all the deliveries associated. The data consists of:

* `name`: the name of the delivery instance;
* `region`: the name of the hub;
* `origin`: the location of the hub with `lat` and `lng`;
* `vehicle_capacity`: the item capacity of the delivery vehicles;
* `deliveries`: list of deliveries, containing dictionaries with the data `id`, `point` (with `lat` and `lng` of delivery) and `size`. 

# 1. Extract, transform and load

Firstly, we extract the desired JSON file to be transformed. The libraries used are Numpy, Pandas and Seaborn. 

In [1]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import json

with open('../input/deliveries-data/deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

len(data), type(data[0])

(199, dict)

We then have a list containing dictionaries of the delivery regions and their respective deliveries to various locations of the city of Brasilia (DF). We can use the first element as an example.

In [4]:
example = data[0]
example.keys()

dict_keys(['name', 'region', 'origin', 'vehicle_capacity', 'deliveries'])

It is possible to visualize our JSON document using the `json` Python package, as follows.

In [5]:
json_formatted_str = json.dumps(example, indent=2)
print(json_formatted_str[0:500])

{
  "name": "cvrp-2-df-33",
  "region": "df-2",
  "origin": {
    "lng": -48.05498915846707,
    "lat": -15.83814451122274
  },
  "vehicle_capacity": 180,
  "deliveries": [
    {
      "id": "313483a19d2f8d65cd5024c8d215cfbd",
      "point": {
        "lng": -48.11618888384239,
        "lat": -15.848929154862294
      },
      "size": 9
    },
    {
      "id": "320c94b17aa685c939b3f3244c3099de",
      "point": {
        "lng": -48.11819489551,
        "lat": -15.850772371049631
      },
      "


## 1.1 - Data Wrangling

We need to put this semi-structured data (JSON format) into a structured table (CSV format) to be able to better manipulate it. For that, we use Pandas to create a DataFrame.

In [6]:
deliveries_df = pd.DataFrame(data)
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


As can be seen, the 'origin' column contains two informations 'lat' and 'lng' that we wish to separate into two individual columns. This procedure is called normalization or flattening, and can be done by the `pd.json_normalize()` method.

In [7]:
hub_origin_df = pd.json_normalize(deliveries_df['origin'])
hub_origin_df.head()

,lng,lat
0,-48.054989,-15.838145
1,-48.054989,-15.838145
2,-48.054989,-15.838145
3,-47.893662,-15.805118
4,-48.054989,-15.838145


We can confirm that there are only three unique values of 'lat' and 'lng' corresponding to the three hub regions.

In [8]:
deliveries_df['region'].unique()

array(['df-2', 'df-1', 'df-0'], dtype=object)

In [9]:
hub_origin_df['lat'].unique()

array([-15.83814451, -15.80511751, -15.65701385])

In [10]:
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df, how='inner', left_index=True, right_index=True)
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries,lng,lat
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145


Now, the 'origin' column can be eliminated, as it's contents have been extracted. We then reorder the columns.

In [11]:
deliveries_df = deliveries_df.drop(['origin'], axis=1)
deliveries_df = deliveries_df[['name', 'region', 'lat', 'lng', 'vehicle_capacity', 'deliveries']]
deliveries_df = deliveries_df.rename(columns={
    'lat': 'hub_lat',
    'lng': 'hub_lng'
})
deliveries_df.head()

,name,region,hub_lat,hub_lng,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,-15.838145,-48.054989,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,-15.838145,-48.054989,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,-15.805118,-47.893662,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,-15.838145,-48.054989,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


Another similar problem occurs with the 'deliveries' column, that has some depth to it. We need to expand the elements of the lists in each row into many rows, then extract the values from the keys of the dictionaries. This process is a **normalization explosion** (expanding the lists into many more rows than before).

In [12]:
deliveries_exploded_df = deliveries_df[['deliveries']].explode('deliveries')
deliveries_exploded_df.head()

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."


Now we have many more rows than the 199 rows of the original DataFrame.

In [13]:
deliveries_exploded_df.shape

(636149, 1)

We then apply some functional programming to extract the desired key-value pairs and create new columns in this DataFrame.

In [14]:
deliveries_exploded_df['delivery_id'] = deliveries_exploded_df['deliveries'].apply(lambda x: x['id'])
deliveries_exploded_df['delivery_point_lng'] = deliveries_exploded_df['deliveries'].apply(lambda x: x['point']['lng'])
deliveries_exploded_df['delivery_point_lat'] = deliveries_exploded_df['deliveries'].apply(lambda x: x['point']['lat'])
deliveries_exploded_df['delivery_size'] = deliveries_exploded_df['deliveries'].apply(lambda x: x['size'])
deliveries_exploded_df.head()

,deliveries,delivery_id,delivery_point_lng,delivery_point_lat,delivery_size
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po...",313483a19d2f8d65cd5024c8d215cfbd,-48.116189,-15.848929,9
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po...",320c94b17aa685c939b3f3244c3099de,-48.118195,-15.850772,2
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po...",3663b42f4b8decb33059febaba46d5c8,-48.112483,-15.847871,1
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi...",e11ab58363c38d6abc90d5fba87b7d7,-48.118023,-15.846471,2
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po...",54cb45b7bbbd4e34e7150900f92d7f4b,-48.114898,-15.858055,7


Then the 'deliveries' column can be dropped because it's contents have been extracted.

In [15]:
deliveries_exploded_df = deliveries_exploded_df.drop(['deliveries'], axis=1)
deliveries_exploded_df.columns

Index(['delivery_id', 'delivery_point_lng', 'delivery_point_lat',
       'delivery_size'],
      dtype='object')

Finally, we merge this new dataframe to the original one using a "right join". This means that every repeated index in the new dataframe receives the rest of the original data copied on many rows. The final dataframe will have 636149 rows.

In [16]:
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_exploded_df, how='right', left_index=True, right_index=True)
deliveries_df = deliveries_df.drop(['deliveries'], axis=1)
deliveries_df.head()

,name,region,hub_lat,hub_lng,vehicle_capacity,delivery_id,delivery_point_lng,delivery_point_lat,delivery_size
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,313483a19d2f8d65cd5024c8d215cfbd,-48.116189,-15.848929,9
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,320c94b17aa685c939b3f3244c3099de,-48.118195,-15.850772,2
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,3663b42f4b8decb33059febaba46d5c8,-48.112483,-15.847871,1
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,e11ab58363c38d6abc90d5fba87b7d7,-48.118023,-15.846471,2
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,180,54cb45b7bbbd4e34e7150900f92d7f4b,-48.114898,-15.858055,7


In [17]:
deliveries_df = deliveries_df.rename(columns={
    'delivery_point_lng': 'delivery_lng',
    'delivery_point_lat': 'delivery_lat'
})
deliveries_df.columns

Index(['name', 'region', 'hub_lat', 'hub_lng', 'vehicle_capacity',
       'delivery_id', 'delivery_lng', 'delivery_lat', 'delivery_size'],
      dtype='object')

In [18]:
deliveries_df.shape

(636149, 9)

Another thing that can be verified is if the resulting dataframe has missing values. In this case, it doesn't have null or nan values, so there's no need for further cleaning of the dataset.

In [19]:
deliveries_df.isna().any()

name                False
region              False
hub_lat             False
hub_lng             False
vehicle_capacity    False
delivery_id         False
delivery_lng        False
delivery_lat        False
delivery_size       False
dtype: bool

The last step is loading this Pandas DataFrame as a CSV file to be stored and used for the rest of the project.

In [20]:
deliveries_df.to_csv('../working/deliveries_clean.csv', sep=',', index=False)


# 2. Reverse Geocoding

**Geocoding** is the process of transforming a location described by text (address, location name etc) into the geographical coordinate (latitude and longitude). In contrast, **reverse geocoding** is the inverse process of converting geographical coordinates into the location described as the address text.

In [21]:
deliveries_df = pd.read_csv('../input/deliveries-data/deliveries_clean.csv', sep=',')
hub_df = deliveries_df[['region', 'hub_lat', 'hub_lng']]
hub_df = hub_df.drop_duplicates().sort_values(by='region').reset_index(drop=True)
hub_df.head()

,region,hub_lat,hub_lng
0,df-0,-15.657014,-47.802665
1,df-1,-15.805118,-47.893662
2,df-2,-15.838145,-48.054989


The Python package `geopy` can process coordinates and return address information. There are free to use geocoding services like **Nominatim**, but there are restrictions such as only allowing one query per second.

In [22]:
import json

import geopy
from geopy.geocoders import Nominatim

lat = str(hub_df['hub_lat'].iloc[0])
lng = str(hub_df['hub_lng'].iloc[0])
coordinate = (lat, lng)
geolocator = Nominatim(user_agent='ebac_geocoder')
location = geolocator.reverse(coordinate) # Accepts tuple of coordinates

print(json.dumps(
    location.raw, indent=2, ensure_ascii=False
))

{
  "place_id": 46082661,
  "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
  "osm_type": "way",
  "osm_id": 240210480,
  "lat": "-15.656916027876347",
  "lon": "-47.80264463632131",
  "class": "highway",
  "type": "secondary",
  "place_rank": 26,
  "importance": 0.10000999999999993,
  "addresstype": "road",
  "name": "Rua 7",
  "display_name": "Rua 7, Quadra 2, Sobradinho, Região Geográfica Imediata do Distrito Federal, Região Integrada de Desenvolvimento do Distrito Federal e Entorno, Região Geográfica Intermediária do Distrito Federal, Distrito Federal, Região Centro-Oeste, 73015-202, Brasil",
  "address": {
    "road": "Rua 7",
    "residential": "Quadra 2",
    "suburb": "Sobradinho",
    "town": "Sobradinho",
    "municipality": "Região Geográfica Imediata do Distrito Federal",
    "county": "Região Integrada de Desenvolvimento do Distrito Federal e Entorno",
    "state_district": "Região Geográfica Intermediária do Distrito Federal",
    "stat

As there are only three instances of hub coordinates, we can extract information about the city and suburb and join it to the DataFrame.

In [23]:
from geopy.extra.rate_limiter import RateLimiter

geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [24]:
# hub_df['coordinates'] = hub_df['hub_lat'].astype(str) + ', ' + hub_df['hub_lng'].astype(str)
hub_df['coordinates'] = hub_df[['hub_lat', 'hub_lng']].apply(tuple, axis=1)
hub_df['geodata'] = hub_df['coordinates'].apply(geocoder)
hub_df.head()

,region,hub_lat,hub_lng,coordinates,geodata
0,df-0,-15.657014,-47.802665,"(-15.657013854445248, -47.802664728268745)","(Rua 7, Quadra 2, Sobradinho, Região Geográfic..."
1,df-1,-15.805118,-47.893662,"(-15.80511751066334, -47.89366206897872)","(SQS 303, Asa Sul, Brasília, Plano Piloto, Reg..."
2,df-2,-15.838145,-48.054989,"(-15.83814451122274, -48.05498915846707)","(Armazém do Bolo, lote 4/8, CSB 4/5, Taguating..."


In [25]:
hub_df['geodata'] = hub_df['geodata'].apply(lambda data: data.raw)
hub_df.head()

,region,hub_lat,hub_lng,coordinates,geodata
0,df-0,-15.657014,-47.802665,"(-15.657013854445248, -47.802664728268745)","{'place_id': 46082661, 'licence': 'Data © Open..."
1,df-1,-15.805118,-47.893662,"(-15.80511751066334, -47.89366206897872)","{'place_id': 46676578, 'licence': 'Data © Open..."
2,df-2,-15.838145,-48.054989,"(-15.83814451122274, -48.05498915846707)","{'place_id': 43508444, 'licence': 'Data © Open..."


Now, we can see that the `geodata` column contains dictionaries. Once again, we need to normalize it via `pd.json_normalize()` to extract the relevant information of city and suburb.

In [26]:
hub_geodata_df = pd.json_normalize(hub_df['geodata'])
hub_geodata_df.head()

,place_id,licence,osm_type,osm_id,lat,lon,class,type,place_rank,importance,...,address.state,address.ISO3166-2-lvl4,address.region,address.postcode,address.country,address.country_code,address.neighbourhood,address.city,address.shop,address.house_number
0,46082661,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,240210480,-15.656916027876347,-47.80264463632131,highway,secondary,26,0.10001,...,Distrito Federal,BR-DF,Região Centro-Oeste,73015-202,Brasil,br,NaN,NaN,NaN,NaN
1,46676578,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,66353368,-15.805172757199022,-47.893723539491205,highway,residential,26,0.10001,...,Distrito Federal,BR-DF,Região Centro-Oeste,70342-010,Brasil,br,SQS 303,Brasília,NaN,NaN
2,43508444,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,6249717596,-15.8384371,-48.0552917,shop,pastry,30,0.00001,...,Distrito Federal,BR-DF,Região Centro-Oeste,72015-030,Brasil,br,NaN,Taguatinga,Armazém do Bolo,lote 4/8


In [27]:
hub_geodata_df.columns

Index(['place_id', 'licence', 'osm_type', 'osm_id', 'lat', 'lon', 'class',
       'type', 'place_rank', 'importance', 'addresstype', 'name',
       'display_name', 'boundingbox', 'address.road', 'address.residential',
       'address.suburb', 'address.town', 'address.municipality',
       'address.county', 'address.state_district', 'address.state',
       'address.ISO3166-2-lvl4', 'address.region', 'address.postcode',
       'address.country', 'address.country_code', 'address.neighbourhood',
       'address.city', 'address.shop', 'address.house_number'],
      dtype='object')

In [28]:
hub_geodata_df = hub_geodata_df[['address.town', 'address.suburb', 'address.city']]
hub_geodata_df.rename(columns={
    'address.town': 'hub_town',
    'address.suburb': 'hub_suburb',
    'address.city': 'hub_city'
}, inplace=True)
hub_geodata_df.head()

,hub_town,hub_suburb,hub_city
0,Sobradinho,Sobradinho,NaN
1,NaN,Asa Sul,Brasília
2,NaN,NaN,Taguatinga


In [29]:
hub_geodata_df['hub_city'] = np.where(hub_geodata_df['hub_city'].notna(), hub_geodata_df['hub_city'], hub_geodata_df['hub_town'])
hub_geodata_df['hub_suburb'] = np.where(hub_geodata_df['hub_suburb'].notna(), hub_geodata_df['hub_suburb'], hub_geodata_df['hub_city'])
hub_geodata_df = hub_geodata_df.drop('hub_town', axis=1)
hub_geodata_df.head()
# Brasília tem subdivisões de bairros, mas não as cidades satélites

,hub_suburb,hub_city
0,Sobradinho,Sobradinho
1,Asa Sul,Brasília
2,Taguatinga,Taguatinga


Com estas informações, podemos combinar o DataFrame gerado com o DataFrame principal `deliveries_df` com o método `pd.merge()`.

In [30]:
hub_df = pd.merge(left=hub_df, right=hub_geodata_df, how='inner', left_index=True, right_index=True)
hub_df = hub_df[['region', 'hub_suburb', 'hub_city']]
hub_df.head()

,region,hub_suburb,hub_city
0,df-0,Sobradinho,Sobradinho
1,df-1,Asa Sul,Brasília
2,df-2,Taguatinga,Taguatinga


In [31]:
deliveries_df = pd.merge(left=deliveries_df, right=hub_df, how='inner', on='region')
deliveries_df = deliveries_df[['name', 'region', 'hub_lat', 'hub_lng', 'hub_suburb', 'hub_city', 'vehicle_capacity', 'delivery_id', 'delivery_size', 'delivery_lat', 'delivery_lng']]
deliveries_df.head()

,name,region,hub_lat,hub_lng,hub_suburb,hub_city,vehicle_capacity,delivery_id,delivery_size,delivery_lat,delivery_lng
0,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,313483a19d2f8d65cd5024c8d215cfbd,9,-15.848929,-48.116189
1,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,320c94b17aa685c939b3f3244c3099de,2,-15.850772,-48.118195
2,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,3663b42f4b8decb33059febaba46d5c8,1,-15.847871,-48.112483
3,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,e11ab58363c38d6abc90d5fba87b7d7,2,-15.846471,-48.118023
4,cvrp-2-df-33,df-2,-15.838145,-48.054989,Taguatinga,Taguatinga,180,54cb45b7bbbd4e34e7150900f92d7f4b,7,-15.858055,-48.114898


## 2.1 - Reverse Geocoding Deliveries

As the `deliveries_clean.csv` file contains 636.149 rows, it would take close to 7 days to transform all the coordinates into addresses using the Nominatim package with one second delays. 

In [32]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv" -O deliveries-geodata.csv